# This notebook creates a model to predict whether or not a give passenger from titanic surived the sinking of the ship

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch','Cabin' ]
X = df[features]
gender_dict = {'male':1, 'female':2}
X['Sex'].replace(gender_dict, inplace = True)
X['Cabin_Block'] = X['Cabin'].str.slice(0,1)
X['Cabin_Block'] = X['Cabin_Block'].fillna('X', inplace = False)
block_dict = {'X': 0, 'C' : 1, 'E' : 2, 'G': 3, 'D' : 4, 'A' : 5, 'B': 6, 'F': 7, 'T': 8}
X['Cabin_Block'].replace(block_dict, inplace = True)
X.drop('Cabin', axis =1, inplace = True)
X = X.fillna(X.mean())
X.isnull().sum()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/cr/8prlvf3s72s0svfyz60fww1m0000gn/T/ipykernel_36617/3699482097.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cabin_Block'] = X['Cabin'].str.slice(0,1)
/var/folders/cr/8prlvf3s72s0svfyz60fww1m0000gn/T/ipykernel_36617/3699482097.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Cabin_Block    0
dtype: int64

In [4]:
X = StandardScaler().fit_transform(X)

In [5]:
y = df['Survived']
n_inputs = X.shape[1]
#normalising data
print('Number of rows in training set is {}'.format(y.shape[0]))
print('number of inouts is {}'.format(X.shape[1]))

Number of rows in training set is 891
number of inouts is 6


In [6]:
#defining a method to create a sequential model
#arguments:
#n_ip - numbers of inputs
#n_hiddenLayers - number of hidden layers in model
#n_hiddenLayerUnits - An array, with ith element specifying the number of units contained in ith hidden layer
def regModel(n_ip,n_hiddenLayers,n_hiddenLayerUnits ):
    model = Sequential()
    model.add(Dense(n_hiddenLayerUnits[0], activation='relu', input_shape=(n_ip,)))
    if n_hiddenLayers > 1:    
        for i in range(1, n_hiddenLayers):   
            model.add(Dense(n_hiddenLayerUnits[i], activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [10]:
#loop around
#mseArr = np.zeros(50)
#acc = np.zeros(50)
#for i in range(0,50):
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
model = regModel(n_inputs, 2, [10, 5])
model.fit(X_train,y_train, epochs= 200, verbose = 0)
y_pred = model.predict(X_test,verbose = 0)
y_pred = np.round(y_pred)
mseArr = mean_squared_error(y_test,y_pred)
acc = (sum(y_pred[:,0] == y_test)*100)/len(y_test)


In [11]:
print('mean of mse is {}'.format(mseArr))
print('mean accuracy is {}'.format(acc))

mean of mse is 0.20522388059701493
mean accuracy is 79.4776119402985


# Adding age to input

In [9]:
data_pred = pd.read_csv('../data/test.csv')
data_pred.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [238]:
X_pred = data_pred[['Pclass','SibSp','Parch','Fare','Sex','Age','Cabin']]
X_pred['Sex'].replace(to_replace='male', value=1, inplace=True)
X_pred['Sex'].replace(to_replace='female', value=2, inplace=True)
X_pred = X_pred.fillna(X_pred.mean())
print('Number of rows in testing set is {}'.format(X_pred.shape[0]))



Number of rows in testing set is 418


In [239]:

#normalising data
X_pred = (X_pred - X_pred.mean())/X_pred.std()
y_pred = model.predict(X_pred)

In [240]:
final_sub = pd.DataFrame()
final_sub['PassengerId'] = data_pred['PassengerId']
final_sub['Survived'] = np.round(y_pred).astype('int')
final_sub.to_csv('../data/submission.csv')